In [1]:
import numpy as np
import matplotlib.pyplot as plt
import imageio.v2 as imageio
import matplotlib.patheffects as path_effects
import os

# Crear carpeta para guardar los frames
output_dir = "frames_pentagon"
os.makedirs(output_dir, exist_ok=True)

# Crear un pentagono regular centrado en el origen
def crear_pentagono(radio=1):
    angulos = np.linspace(0, 2 * np.pi, 6)[:-1]  # 5 lados
    x = radio * np.cos(angulos)
    y = radio * np.sin(angulos)
    x = np.append(x, x[0])
    y = np.append(y, y[0])
    return np.vstack((x, y))  # shape (2, N)

# Matrices de transformacion
def matriz_traslacion(tx, ty):
    return np.array([[1, 0, tx],
                     [0, 1, ty],
                     [0, 0, 1]])

def matriz_rotacion(theta):
    c, s = np.cos(theta), np.sin(theta)
    return np.array([[c, -s, 0],
                     [s,  c, 0],
                     [0,  0, 1]])

def matriz_escala(sx, sy):
    return np.array([[sx, 0,  0],
                     [0,  sy, 0],
                     [0,  0,  1]])

# Convertir coordenadas a homogeneas
def a_homogeneas(puntos_2d):
    unos = np.ones((1, puntos_2d.shape[1]))
    return np.vstack((puntos_2d, unos))  # shape (3, N)

# Aplicar transformacion
def aplicar_transformacion(M, puntos_homog):
    resultado = M @ puntos_homog
    return resultado[:2, :]  # eliminar coordenada homogenea

# Figura base
pentagono = crear_pentagono()
pentagono_homog = a_homogeneas(pentagono)

# Parametros
num_frames = 45
centro_rotacion = np.array([2, 2])
trayectoria_centro = []  # almacenara la posicion del centro transformado

cuadros = []

for t in range(num_frames):
    progreso = t / num_frames
    angulo = 2 * np.pi * progreso
    escala = 1 + 0.5 * np.sin(2 * np.pi * progreso)

    # Transformaciones
    T1 = matriz_traslacion(-centro_rotacion[0], -centro_rotacion[1])
    R = matriz_rotacion(angulo)
    S = matriz_escala(escala, escala)
    T2 = matriz_traslacion(centro_rotacion[0], centro_rotacion[1])
    M = T2 @ R @ S @ T1

    figura = aplicar_transformacion(M, pentagono_homog)

    # Calcular el nuevo centro del pentagono (transformado)
    centro_original = np.array([[0], [0]])  # el centro original esta en el origen
    centro_homog = a_homogeneas(centro_original)
    centro_actual = aplicar_transformacion(M, centro_homog).flatten()
    trayectoria_centro.append(centro_actual)

    # Graficar
    fig, ax = plt.subplots(figsize=(6, 6))
    ax.plot(*figura, color="purple", linewidth=2)
    ax.fill(*figura, color="purple", alpha=0.3)

    # Traza de trayectoria
    if t > 0:
        tray = np.array(trayectoria_centro).T
        ax.plot(tray[0], tray[1], color='deepskyblue', linestyle='--', linewidth=1.5, label="Trayectoria")

    # Visuales
    ax.grid(True, which='both', linestyle='--', linewidth=0.5)
    ax.axhline(0, color='black', linewidth=0.5)
    ax.axvline(0, color='black', linewidth=0.5)
    ax.set_xlim(-4, 8)
    ax.set_ylim(-4, 8)
    ax.set_aspect('equal')
    ax.set_title(f"Frame {t+1}/{num_frames}", fontsize=10)
    ax.scatter(*centro_rotacion, color="red", s=30, label="Centro de rotación")
    ax.scatter(*centro_actual, color="blue", s=20, label="Centro actual")
    ax.legend(loc="upper right", fontsize=8)

    # Mostrar matriz M en la esquina inferior izquierda
    M_text = "\n".join(["  ".join(f"{val:6.2f}" for val in fila) for fila in M])
    text_obj = ax.text(-3.8, -3.5, f"Matriz de transformación:\n{M_text}",
                       fontsize=8, family='monospace', va='bottom', ha='left', color='white',
                       bbox=dict(boxstyle="round,pad=0.3", facecolor='black', alpha=0.5))

    text_obj.set_path_effects([path_effects.Stroke(linewidth=1, foreground='black'),
                               path_effects.Normal()])

    nombre_cuadro = f"{output_dir}/cuadro_{t:03d}.png"
    plt.savefig(nombre_cuadro)
    plt.close()
    cuadros.append(nombre_cuadro)

with imageio.get_writer("pentagono_animado.gif", mode='I', duration=0.05) as writer:
    for cuadro in cuadros:
        imagen = imageio.imread(cuadro)
        writer.append_data(imagen)